In [4]:
import re
from nipype import Node
from nipype.interfaces.io import DataFinder

In [5]:
# First get the folder names for the masks and tracks
seedMaskFinder = Node(DataFinder(match_regex = '.*seed.*mask.*\.nii\.gz'), name = 'seedMaskFinder')
targetMaskFinder = Node(DataFinder(match_regex = '.*target*.mask.*\.nii\.gz'), name = 'seedMaskFinder')
trackFolderFinder = Node(DataFinder(match_regex = '/*track/*', unpack_single = True), name = 'trackFolderFinder')

In [63]:
tracking_dir = '/Users/srothmei/Desktop/charite/toronto/FR_20120903/tractography/'
seedMaskFinder.inputs.root_paths = tracking_dir
seedmasks = seedMaskFinder.run()
targetMaskFinder.inputs.root_paths = tracking_dir
targetmasks = targetMaskFinder.run()
trackFolderFinder.inputs.root_paths = tracking_dir
trackFolder = trackFolderFinder.run().outputs.out_paths

# Build subset
subSet = seedmasks.outputs.out_paths[0:10]
subSet = dict(zip(subSet, range(len(subSet))))
#print subSet

for i in subSet.keys():
    # Get the mask id i.e. the last chain of min. 4 digits in the current string
    maskID = re.findall("\d{4,999}", i)[-1][:4]
    # Now find the corresponding mask in the set of possible target masks
    matching = [s for s in targetmasks.outputs.out_paths if maskID in s]
    # Insert into dict
    subSet[i] = matching[0]

print subSet


INFO:workflow:Executing node seedMaskFinder in dir: /var/folders/tg/2g576vk174s3m9znw1sgjpbm0000gn/T/tmp1arpPM/seedMaskFinder
INFO:workflow:Executing node seedMaskFinder in dir: /var/folders/tg/2g576vk174s3m9znw1sgjpbm0000gn/T/tmpqogXZZ/seedMaskFinder
INFO:workflow:Executing node trackFolderFinder in dir: /var/folders/tg/2g576vk174s3m9znw1sgjpbm0000gn/T/tmpPmPPWD/trackFolderFinder


{'/Users/srothmei/Desktop/charite/toronto/FR_20120903/tractography/masks/seedmask10011_1mm.nii.gz': '/Users/srothmei/Desktop/charite/toronto/FR_20120903/tractography/masks/targetmask1001_1mm.nii.gz', '/Users/srothmei/Desktop/charite/toronto/FR_20120903/tractography/masks/seedmask10034_1mm.nii.gz': '/Users/srothmei/Desktop/charite/toronto/FR_20120903/tractography/masks/targetmask1003_1mm.nii.gz', '/Users/srothmei/Desktop/charite/toronto/FR_20120903/tractography/masks/seedmask10035_1mm.nii.gz': '/Users/srothmei/Desktop/charite/toronto/FR_20120903/tractography/masks/targetmask1003_1mm.nii.gz', '/Users/srothmei/Desktop/charite/toronto/FR_20120903/tractography/masks/seedmask10022_1mm.nii.gz': '/Users/srothmei/Desktop/charite/toronto/FR_20120903/tractography/masks/targetmask1002_1mm.nii.gz', '/Users/srothmei/Desktop/charite/toronto/FR_20120903/tractography/masks/seedmask10021_1mm.nii.gz': '/Users/srothmei/Desktop/charite/toronto/FR_20120903/tractography/masks/targetmask1002_1mm.nii.gz', '/Us

In [46]:
res = re.findall("\d{4,999}", foo[0][1])[-1]
print res

10011


In [20]:
import os, re

In [31]:
theDir = '/Users/srothmei/Desktop/charite/toronto/AJ_20140516_1600/mrtrix_68/tracks_68/'
distFiles = list()
capfiles = list()
for fileZ in os.listdir(theDir):
    if re.search("^SC_cap_row*.", fileZ):
        capfiles.append(theDir + fileZ)
    elif re.search("^SC_dist_row*.", fileZ):
        distFiles.append(theDir + fileZ)

In [32]:
capfiles.sort()
distFiles.sort()
for cap_file, dist_file in zip(capfiles, distFiles):
    print dist_file

/Users/srothmei/Desktop/charite/toronto/AJ_20140516_1600/mrtrix_68/tracks_68/SC_dist_row_10AJ_20140516_1600.json
/Users/srothmei/Desktop/charite/toronto/AJ_20140516_1600/mrtrix_68/tracks_68/SC_dist_row_11AJ_20140516_1600.json
/Users/srothmei/Desktop/charite/toronto/AJ_20140516_1600/mrtrix_68/tracks_68/SC_dist_row_12AJ_20140516_1600.json
/Users/srothmei/Desktop/charite/toronto/AJ_20140516_1600/mrtrix_68/tracks_68/SC_dist_row_13AJ_20140516_1600.json
/Users/srothmei/Desktop/charite/toronto/AJ_20140516_1600/mrtrix_68/tracks_68/SC_dist_row_14AJ_20140516_1600.json
/Users/srothmei/Desktop/charite/toronto/AJ_20140516_1600/mrtrix_68/tracks_68/SC_dist_row_15AJ_20140516_1600.json
/Users/srothmei/Desktop/charite/toronto/AJ_20140516_1600/mrtrix_68/tracks_68/SC_dist_row_16AJ_20140516_1600.json
/Users/srothmei/Desktop/charite/toronto/AJ_20140516_1600/mrtrix_68/tracks_68/SC_dist_row_17AJ_20140516_1600.json
/Users/srothmei/Desktop/charite/toronto/AJ_20140516_1600/mrtrix_68/tracks_68/SC_dist_row_18AJ_20